In [1]:
import json
import requests
import pandas as pd
import re
import math

In [4]:
api_key = 'QvkORZTgzyx1-ZtFlig2u6fyqQpbGVnNMGz2kSGtAqzml3QViP4lpQcZinGG0ahJXkpoUZzjjKgIfSDSRWL_ejcx1tB2h_CrWqMLFYRtKryNvWiCjENE4YmP9jFqYHYx'

# verifies connection to yelp db using initial call

In [5]:
top_cities = ["New York City", 
              "Los Angeles",
              "Chicago",
              "Houston",
              "Phoenix",
              "Philadelphia",
              "San Antonio",
              "San Diego",
              "Dallas",
              "Austin",
              "San Jose",
              "Fort Worth",
              "Jacksonville",
              "Columbus",
              "Charlotte",
              "Indianapolis",
              "San Francisco",
              "Seattle",
              "Denver",
              "Washington"]

In [200]:
#city_name = 'Fort Worth'

#city_name = 'Jacksonville'

#city_name = 'Columbus'

#city_name = 'Charlotte'

#city_name = 'Indianapolis'

#city_name = 'San Francisco'

#city_name = 'Denver'

#city_name = 'Washington'

#city_name = 'Chicago'

#city_name = 'Austin'

city_name = 'New York City'

In [201]:
headers = {'Authorization': 'Bearer %s' % api_key}
url = 'https://api.yelp.com/v3/businesses/search'

params = {'location':city_name, 
          'offset':500, 
          'limit':50, 
          'categories':'restaurants'}
req = requests.get(url, params=params, headers=headers)
print('The status code is {}'.format(req.status_code))
city_businesses = json.loads(req.text)
print(f'{city_name}, {city_businesses["total"]}')

The status code is 200
New York City, 18800


## total businesses returned, can only take first 1k

In [202]:
city_businesses = json.loads(req.text)
city_businesses['total']

18800

# Previous code to find out which restaurant types exist in seattle

In [203]:
yelp_types = pd.read_csv('yelp_restaurant_types.csv', header=None)

In [204]:
total_bus = []
for rtype in yelp_types[0]:
    params = {'location': city_name, 
              'offset':0, 
              'limit':50, 
              'categories':rtype}
    req = requests.get(url, params=params, headers=headers)
    api_results = json.loads(req.text)   
    total_bus.append(api_results['total'])

In [205]:
api_results

{'businesses': [],
 'total': 0,
 'region': {'center': {'longitude': -73.99429321289062,
   'latitude': 40.70544486444615}}}

In [206]:
found_bus_types = pd.DataFrame(zip(yelp_types[0].values, total_bus))
found_bus_types

,0,1
0,afghani,19
1,african,49
2,senegalese,8
3,southafrican,1
4,newamerican,1100
...,...,...
307,vietnamese,136
308,waffles,39
309,wok,0
310,wraps,82


In [207]:
found_bus_types[found_bus_types[1] == found_bus_types[1].max()]

,0,1
239,pizza,1800
262,sandwiches,1800


In [208]:
found_bus_types[1].sum()

30731

# Loading previously queried restaurant types in seattle

In [209]:
all_types = pd.read_csv(r'yelp_restaurant_types.csv', header=None)
all_types = all_types[0]
# type_exists = found_bus_types[found_bus_types[1] != 0][0]

## Looping through every restaurant type

In [210]:
valid_rest = found_bus_types[found_bus_types[1] != 0]

In [211]:
city_name

'New York City'

In [212]:
rest_results = []
for rtype in valid_rest[0]:
    num_rests = valid_rest[valid_rest[0] == rtype][1].values[0]
    loops = math.ceil(num_rests/50)
    print(rtype, num_rests, loops)
    
    for n in range(loops):
        try:
            params = {'location':city_name, 
                      'offset':50*n, 
                      'limit':50, 
                      'categories': rtype}
            req = requests.get(url, params=params, headers=headers)
            api_results = json.loads(req.text)   
            rest_results = rest_results + api_results['businesses']
        except Exception as e:
            print(f'{e} occured at {city_name}, {rtype}, offset: {n}')

compiled_df = pd.DataFrame(rest_results)
compiled_df = compiled_df.drop_duplicates('id').reset_index(drop=True)
compiled_df['longitude'] = compiled_df['coordinates'].str['longitude']
compiled_df['latitude'] = compiled_df['coordinates'].str['latitude']
compiled_df['zip_code'] = compiled_df['location'].str['zip_code']
compiled_df.to_csv(f'{city_name}_yelp_data.csv', index=False)

afghani 19 1
african 49 1
senegalese 8 1
southafrican 1 1
newamerican 1100 22
'businesses' occured at New York City, newamerican, offset: 20
'businesses' occured at New York City, newamerican, offset: 21
tradamerican 1100 22
'businesses' occured at New York City, tradamerican, offset: 20
'businesses' occured at New York City, tradamerican, offset: 21
arabian 5 1
argentine 22 1
armenian 5 1
asianfusion 333 7
australian 15 1
austrian 5 1
bangladeshi 35 1
bbq 235 5
'businesses' occured at New York City, bbq, offset: 2
'businesses' occured at New York City, bbq, offset: 3
'businesses' occured at New York City, bbq, offset: 4
basque 12 1
'businesses' occured at New York City, basque, offset: 0
belgian 15 1
'businesses' occured at New York City, belgian, offset: 0
bistros 1 1
'businesses' occured at New York City, bistros, offset: 0
brasseries 15 1
'businesses' occured at New York City, brasseries, offset: 0
brazilian 43 1
'businesses' occured at New York City, brazilian, offset: 0
breakfast

In [213]:
compiled_df['id'].unique().shape

(2554,)

In [214]:
# api_results

# Formatting results

In [215]:
#compiled_df = pd.read_csv(r'seattle_yelp_data.csv')
compiled_df = pd.DataFrame(rest_results)
compiled_df = compiled_df.drop_duplicates('id').reset_index(drop=True)

## Parsing coordinates

## if using query results

In [216]:
compiled_df['longitude'] = compiled_df['coordinates'].str['longitude']
compiled_df['latitude'] = compiled_df['coordinates'].str['latitude']
compiled_df['zip_code'] = compiled_df['location'].str['zip_code']

## if loading from csv

### parsing coordinates

In [217]:
# coords = compiled_df['coordinates'].str.split(',', expand=True)
# compiled_df.insert(0, 'longitude', coords[1])
# compiled_df.insert(0, 'latitude', coords[0])
# compiled_df['longitude'] = compiled_df['longitude'].str.split(' ').str[-1].str[:-1]
# compiled_df['latitude'] = compiled_df['latitude'].str.split(' ').str[-1]

### Parsing zip code

In [218]:
# r = re.compile("code': '(\d*)")
# compiled_df['zip_code'] = compiled_df['location'].str.findall(r).str[0]

## Output

In [219]:
compiled_df.shape

(2554, 19)

In [220]:
compiled_df.to_csv(f'{city_name}_yelp_data.csv', index=False)

_____________

# Code for wrangling the restaurant types from yelp page, not needed anymore

In [131]:
# id_list = []
# name_list = []
# for i in sea_bus:
#     id_list.append(i['id'])
#     name_list.append(i['name'])

In [153]:
# rest_types = re.findall("\([a-z].*?,", rest_str)

In [164]:
# len(cleaned_types)

312

In [25]:
# """
# Afghan (afghani)
# African (african)
# Senegalese (senegalese)
# South African (southafrican)
# American (New) (newamerican)
# American (Traditional) (tradamerican)
# Arabian (arabian)
# Argentine (argentine)
# Armenian (armenian)
# Asian Fusion (asianfusion)
# Australian (australian)
# Austrian (austrian)
# Bangladeshi (bangladeshi)
# Barbeque (bbq)
# Basque (basque)
# Belgian (belgian)
# Brasseries (brasseries)
# Brazilian (brazilian)
# Breakfast & Brunch (breakfast_brunch)
# Pancakes (pancakes)
# British (british)
# Buffets (buffets)
# Bulgarian (bulgarian)
# Burgers (burgers)
# Burmese (burmese)
# Cafes (cafes)
# Themed Cafes (themedcafes)
# Cafeteria (cafeteria)
# Cajun/Creole (cajun)
# Cambodian (cambodian)
# Caribbean (caribbean)
# Dominican (dominican)
# Haitian (haitian)
# Puerto Rican (puertorican)
# Trinidadian (trinidadian)
# Catalan (catalan)
# Cheesesteaks (cheesesteaks)
# Chicken Shop (chickenshop)
# Chicken Wings (chicken_wings)
# Chinese (chinese)
# Cantonese (cantonese)
# Dim Sum (dimsum)
# Hainan (hainan)
# Shanghainese (shanghainese)
# Szechuan (szechuan)
# Comfort Food (comfortfood)
# Creperies (creperies)
# Cuban (cuban)
# Czech (czech)
# Delis (delis)
# Diners (diners)
# Dinner Theater (dinnertheater)
# Eritrean (eritrean)
# Ethiopian (ethiopian)
# Fast Food (hotdogs)
# Filipino (filipino)
# Fish & Chips (fishnchips)
# Fondue (fondue)
# Food Court (food_court)
# Food Stands (foodstands)
# French (french)
# Mauritius (mauritius)
# Reunion (reunion)
# Game Meat (gamemeat)
# Gastropubs (gastropubs)
# Georgian (georgian)
# German (german)
# Gluten-Free (gluten_free)
# Greek (greek)
# Guamanian (guamanian)
# Halal (halal)
# Hawaiian (hawaiian)
# Himalayan/Nepalese (himalayan)
# Honduran (honduran)
# Hong Kong Style Cafe (hkcafe)
# Hot Dogs (hotdog)
# Hot Pot (hotpot)
# Hungarian (hungarian)
# Iberian (iberian)
# Indian (indpak)
# Indonesian (indonesian)
# Irish (irish)
# Italian (italian)
# Calabrian (calabrian)
# Sardinian (sardinian)
# Sicilian (sicilian)
# Tuscan (tuscan)
# Japanese (japanese)
# Conveyor Belt Sushi (conveyorsushi)
# Izakaya (izakaya)
# Japanese Curry (japacurry)
# Ramen (ramen)
# Teppanyaki (teppanyaki)
# Kebab (kebab)
# Korean (korean)
# Kosher (kosher)
# Laotian (laotian)
# Latin American (latin)
# Colombian (colombian)
# Salvadoran (salvadoran)
# Venezuelan (venezuelan)
# Live/Raw Food (raw_food)
# Malaysian (malaysian)
# Mediterranean (mediterranean)
# Falafel (falafel)
# Mexican (mexican)
# Tacos (tacos)
# Middle Eastern (mideastern)
# Egyptian (egyptian)
# Lebanese (lebanese)
# Modern European (modern_european)
# Mongolian (mongolian)
# Moroccan (moroccan)
# New Mexican Cuisine (newmexican)
# Nicaraguan (nicaraguan)
# Noodles (noodles)
# Pakistani (pakistani)
# Pan Asian (panasian)
# Persian/Iranian (persian)
# Peruvian (peruvian)
# Pizza (pizza)
# Polish (polish)
# Polynesian (polynesian)
# Pop-Up Restaurants (popuprestaurants)
# Portuguese (portuguese)
# Poutineries (poutineries)
# Russian (russian)
# Salad (salad)
# Sandwiches (sandwiches)
# Scandinavian (scandinavian)
# Scottish (scottish)
# Seafood (seafood)
# Singaporean (singaporean)
# Slovakian (slovakian)
# Somali (somali)
# Soul Food (soulfood)
# Soup (soup)
# Southern (southern)
# Spanish (spanish)
# Sri Lankan (srilankan)
# Steakhouses (steak)
# Supper Clubs (supperclubs)
# Sushi Bars (sushi)
# Syrian (syrian)
# Taiwanese (taiwanese)
# Tapas Bars (tapas)
# Tapas/Small Plates (tapasmallplates)
# Tex-Mex (tex-mex)
# Thai (thai)
# Turkish (turkish)
# Ukrainian (ukrainian)
# Uzbek (uzbek)
# Vegan (vegan)
# Vegetarian (vegetarian)
# Vietnamese (vietnamese)
# Waffles (waffles)
# Wraps (wraps)
# """

'\nAfghan (afghani)\nAfrican (african)\nSenegalese (senegalese)\nSouth African (southafrican)\nAmerican (New) (newamerican)\nAmerican (Traditional) (tradamerican)\nArabian (arabian)\nArgentine (argentine)\nArmenian (armenian)\nAsian Fusion (asianfusion)\nAustralian (australian)\nAustrian (austrian)\nBangladeshi (bangladeshi)\nBarbeque (bbq)\nBasque (basque)\nBelgian (belgian)\nBrasseries (brasseries)\nBrazilian (brazilian)\nBreakfast & Brunch (breakfast_brunch)\nPancakes (pancakes)\nBritish (british)\nBuffets (buffets)\nBulgarian (bulgarian)\nBurgers (burgers)\nBurmese (burmese)\nCafes (cafes)\nThemed Cafes (themedcafes)\nCafeteria (cafeteria)\nCajun/Creole (cajun)\nCambodian (cambodian)\nCaribbean (caribbean)\nDominican (dominican)\nHaitian (haitian)\nPuerto Rican (puertorican)\nTrinidadian (trinidadian)\nCatalan (catalan)\nCheesesteaks (cheesesteaks)\nChicken Shop (chickenshop)\nChicken Wings (chicken_wings)\nChinese (chinese)\nCantonese (cantonese)\nDim Sum (dimsum)\nHainan (hainan)